In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
class SVMClassifier: 
    
    def __init__(self, data_path, selected_features, label, kernel):
        self.data_path = data_path
        self.selected_features = selected_features
        self.label = label
        self.kernel = kernel
        self.model = SVC(kernel = kernel, gamma='scale')
    
    def _load_data(self):
        self.df = pd.read_csv(self.data_path)
        self.df['label'] = self.df['label'].apply(lambda x: x if x == 'normal.' else 'attack')

    
    def data_preparation(self):
        X = self.df[self.selected_features]
        y = self.df[self.label]
        X_train, X_test, y_train, y_test = train_test_split(self.df[selected_features], self.df['label'], test_size=0.2, random_state=42)
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
    def train_model(self):
        t0 = time.time()
        self.model.fit(self.X_train, self.y_train)
        training_time = time.time() - t0
        
    def evaluate_model(self):
        y_pred = self.model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        conf_matrix = confusion_matrix(self.y_test, y_pred)
        f1 = f1_score(self.y_test, y_pred, average='weighted')
        class_report = classification_report(self.y_test, y_pred)
        return accuracy, conf_matrix, f1, class_report

In [3]:
selected_features = ['dst_host_same_src_port_rate',
 'is_guest_login',
 'num_failed_logins',
 'dst_host_diff_srv_rate',
 'rerror_rate',
 'logged_in',
 'serror_rate',
 'same_srv_rate',
 'protocol_type_0',
'protocol_type_1', 
 'dst_host_srv_diff_host_rate']

In [4]:
instance = SVMClassifier("../../../transformed_datasets/transformed_KDD_CUPP99_2.csv", selected_features, 'label', 'rbf')

In [5]:
instance._load_data()

In [6]:
instance.data_preparation()

In [7]:
instance.train_model()

KeyboardInterrupt: 

In [ ]:
instance.evaluate_model()